# Decides which patients may be included in the modelling


Criteria:
1. an admission is only eligible if there are diagnostics associated with it
2. Patient must have more than 1 (eligible) admission to the hospital

In [10]:
from Mimic import Mimic
from ICDCodesGrouper import ICDCodesGrouper
import pandas as pd
import numpy as np

In [3]:
grouper = ICDCodesGrouper()
mimic = Mimic(grouper=grouper)

In [19]:
admissions = mimic.read_admissions()
diagnoses = mimic.read_diagnoses()

# 1. remove admissions without record of diagnoses
admissions_without_records= diagnoses.loc[pd.isna(diagnoses.ICD9_CODE),'HADM_ID'].unique()
admissions = admissions[~admissions.HADM_ID.isin(admissions_without_records)]
# 2. Only interested in patients with multiple admissions
mult_adm_patients = admissions.groupby('SUBJECT_ID').size().where(lambda x: x > 1).dropna().index.tolist()

# show 5 patients (id)
print('nº patients eligible:',len(mult_adm_patients))

nº patients eligible: 7499


# Save eligible patients

In [15]:
np.savetxt('data/eligible_patients.txt', mult_adm_patients, fmt='%d',delimiter=',')